# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 13 2023 10:54AM,255004,10,0086298072,ISDIN Corporation,Released
1,Jan 13 2023 10:54AM,255004,10,0086298073,ISDIN Corporation,Released
2,Jan 13 2023 10:54AM,255004,10,0086298083,ISDIN Corporation,Released
3,Jan 13 2023 10:54AM,255004,10,0086298084,ISDIN Corporation,Released
4,Jan 13 2023 10:54AM,255004,10,0086298131,ISDIN Corporation,Released
5,Jan 13 2023 10:52AM,255003,19,9018723,MedStone Pharma LLC,Released
6,Jan 13 2023 10:39AM,255002,19,ADV80010273,"AdvaGen Pharma, Ltd",Released
7,Jan 13 2023 10:07AM,254999,10,9023067,Eywa Pharma Inc.,Released
8,Jan 13 2023 10:07AM,254999,10,9014398-BO,Eywa Pharma Inc.,Released
9,Jan 13 2023 10:07AM,254999,10,9024349,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,254997,Released,21
20,254999,Released,6
21,255002,Released,1
22,255003,Released,1
23,255004,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254997,NaN,21.0
254999,NaN,6.0
255002,NaN,1.0
255003,NaN,1.0
255004,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254918,0.0,1.0
254922,1.0,0.0
254949,0.0,4.0
254961,1.0,0.0
254976,7.0,41.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254918,0,1
254922,1,0
254949,0,4
254961,1,0
254976,7,41


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254918,0,1
1,254922,1,0
2,254949,0,4
3,254961,1,0
4,254976,7,41


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254918,,1
1,254922,1,
2,254949,,4
3,254961,1,
4,254976,7,41


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation
5,Jan 13 2023 10:52AM,255003,19,MedStone Pharma LLC
6,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd"
7,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.
13,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC"
18,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC"
19,Jan 13 2023 9:45AM,254997,10,Bio-PRF
40,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc."
109,Jan 13 2023 9:44AM,254992,15,"Alliance Pharma, Inc."
117,Jan 13 2023 9:37AM,254993,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,,5
1,Jan 13 2023 10:52AM,255003,19,MedStone Pharma LLC,,1
2,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",,1
3,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,,6
4,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",,5
5,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",,1
6,Jan 13 2023 9:45AM,254997,10,Bio-PRF,,21
7,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc.",39,30
8,Jan 13 2023 9:44AM,254992,15,"Alliance Pharma, Inc.",,8
9,Jan 13 2023 9:37AM,254993,10,"Citieffe, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,5,
1,Jan 13 2023 10:52AM,255003,19,MedStone Pharma LLC,1,
2,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1,
3,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6,
4,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5,
5,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",1,
6,Jan 13 2023 9:45AM,254997,10,Bio-PRF,21,
7,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc.",30,39
8,Jan 13 2023 9:44AM,254992,15,"Alliance Pharma, Inc.",8,
9,Jan 13 2023 9:37AM,254993,10,"Citieffe, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,5,
1,Jan 13 2023 10:52AM,255003,19,MedStone Pharma LLC,1,
2,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1,
3,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6,
4,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,5.0,NaN
1,Jan 13 2023 10:52AM,255003,19,MedStone Pharma LLC,1.0,NaN
2,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1.0,NaN
3,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6.0,NaN
4,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,5.0,0.0
1,Jan 13 2023 10:52AM,255003,19,MedStone Pharma LLC,1.0,0.0
2,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",1.0,0.0
3,Jan 13 2023 10:07AM,254999,10,Eywa Pharma Inc.,6.0,0.0
4,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2804862,53.0,5.0
12,254961,0.0,1.0
15,1019977,44.0,39.0
19,764981,43.0,7.0
21,509840,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2804862,53.0,5.0
1,12,254961,0.0,1.0
2,15,1019977,44.0,39.0
3,19,764981,43.0,7.0
4,21,509840,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,53.0,5.0
1,12,0.0,1.0
2,15,44.0,39.0
3,19,43.0,7.0
4,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,53.0
1,12,Released,0.0
2,15,Released,44.0
3,19,Released,43.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Executing,5.0,1.0,39.0,7.0,1.0
Released,53.0,0.0,44.0,43.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Executing,5.0,1.0,39.0,7.0,1.0
1,Released,53.0,0.0,44.0,43.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Executing,5.0,1.0,39.0,7.0,1.0
1,Released,53.0,0.0,44.0,43.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()